
## Using fastai on Clinical Time Series data

I dette prosjektet skal jeg bruke deep learning for å predikere dødlighet under et sykehus besøk på intensivavdeling. Prosjektet er først og fremst et læringsprosjekt som en forberedelse på min masteroppgave. Deep learning delen bruker fastai rammeverket (http://www.fast.ai/)

Dataen brukt i prosjektet er MIMIC III datasettet (https://www.nature.com/articles/sdata201635). Denne dataen er innhentet Beth Israel Deaconess Medical Center mellom 2001 og 2012. Videre er dataen anonymisert slik at den kan brukes til forskning og data analyse. Dataen er delt opp i flere CSV filer, hvor jeg har brukt filene “stays”, “diagnoses”, “patients”, “admissions”, “labevents”, “chartevents” og “outputevents”.

Dette prosjektet er basert på artikkelen "Multitask Learning and Benchmarking with Clinical Time Series Data" skrevet av Hrayr Harutyunyan, Hrant Khachatrian, David C. Kale, og Aram Galstyan (https://arxiv.org/abs/1703.07771)  på YerevaNNs forsknings lab. Dataprossesseringen og valg av trening, validering og test data er basert på deres arbeid. (https://github.com/YerevaNN/mimic3-benchmarks). 

Det beste resultatet de oppnådde var: 
- AUC of ROC = 0.8625
- AUC of PRC = 0.5169
- min(+P, Se) = 0.4987

Ettersom dataprossesseringen er basert på deres prosjekt, går jeg ikke inn i detaljer her. Men raskt forklart blir dataen delt opp for hver enkelt pasient, hvor den så blir omgjort til episode data. Underveis blir pasienter under 18 og pasienter som har vært innom flere intensivavdelinger filtrert bort. 

For å fylle ut manglende data, har jeg skrevet en "Imputer" som er basert på fremgangsmåten gjort på YerevaNN. Den fyller inn manglende verdier utifra forrige måling. Mangler det verdier i første kolonne, brukes det standardverdier. 
    
    
Jeg har prøvd to ulike fremgangsmåter for å trene nettverket: 

### Fremgangsmåte 1
Her ble all episodedata for alle pasienter lagt sammen til ett stort datasett. 

Resultatene gitt av fremgangsmåte 1 er relativt dårlige( se fil: 5.0-train_model). 

Siste resultat ble: 

- AUC of ROC = 0.5070665964117684
- AUC of PRC = 0.1935953643172068
- min(+P, Se) = 0.125



### Fremgangsmåte 2 
En anderledes måte å representere dataen. Hver episode blir endret til å være 1 rad med data.
Eksempel er en episode med 38 rader. Her vil "hearth rate" bli representert ved "hearth rate0", "hearth rate1" .... "hearth rate38". 

Ettersom episodene er av ulike lengder og ett nevralt nettverk krever samme antall parametre på all data, må man velge en felles lengde for all data. Dermed må man ta noen valg: 


- All data som har lengde mindre enn 30 blir fjernet. 

- Data mellom 25 og 100 blir den første rader kopiert frem til den har lengde på 100. 

- Data over 100 ser man bare på de 100 siste radene. 

- Orginal lengde blir lagt til som en variabel i datasettet.


Det beste resultatet her(hittil i hvertfall) ble: 
- AUC of ROC = 0.6754233044466119
- AUC of PRC = 0.47102764943984265
- min(+P, Se) = 0.4144385026737968

For mer detaljer se fil; "6.0 - Make_patient_to_single_row_data.ipynb" for dataprossesseringen og fil; "7.0 - train_model_with_row_data" for treningen. 


### Embeddings
Begge fremgangsmåtene brukte "embeddings" for å håndtere kategoriske verdier. Embeddings tilbyr en rikere fremstilling av data enn one hot encoding, noe som kan føre til bedre resultater. 
Dette vil bli forklart bedre under eksamen. 

For å definere størrelsen på embeddings brukes formelen gitt av Jeremy Howard:
    emb_size = min(50, 1+(c/2)). I formelen er c antall kategorier i den kategoriske dataen. 


### Videre arbeid
Selv med fremgangsmåte 2 er resultatet dårligere enn fra YerevaNN. Det er mye man kan prøve ut for å videre forbedre modellen min. 
- Prøve ut gjennomsnittlige verdier for å fylle ut tomme verdier.  
- Prøve ut ulike lengder for episode data. 
- tweaking av hyperparametre. 




